# Collect data for github action workflow runs

In this notebook, we collect historical test data like the test duration values from running workflows on Github using the GitHub API

## Collect data for selected workflow runs of a repository

From historical test workflow runs, want to extract
- time durations
- workflow run status & conclusion

We can get workflow IDs of the test that we are interested in from https://api.github.com/repos/{ORG}/{REPO}/actions/workflows 

In [1]:
from dotenv import find_dotenv, load_dotenv
import os
import json
import subprocess
from datetime import datetime
from subprocess import PIPE
import pandas as pd
pd.options.mode.chained_assignment = None

import warnings

warnings.filterwarnings("ignore")

In [2]:
load_dotenv(find_dotenv(), override=True)
TOKEN = os.getenv("GITHUB_ACCESS_TOKEN")
ORG = os.getenv("ORG")
REPO = os.getenv("REPO")

In [3]:
# mode can be "check or "workflow" depending on whether you want to collect checks data or workflows data
MODE = "check"

* Workflows: For example, lets collect data for the test ID 28698040 for the workflow runs in the repository `oss-aspen/8Knot` https://api.github.com/repos/oss-aspen/8Knot/actions/workflows

* Checks: Get commits for a repo and get check runs for each commit

In [4]:
def get_page_numbers(test_id):
    """
    Get the total count of tests.
    Find the pages on github-actions.
    """
    command = """curl \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/actions/workflows/{}/runs?""".format(TOKEN, ORG, REPO, test_id)
    args = []
    args.append(command)
    output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
    output = json.loads(output.stdout)
    total_count = output['total_count']
    page_numbers = int(total_count/30) # by default number of tests on one page is 30
    return page_numbers

In [5]:
def get_workflow_runs(test_id, page_numbers):
    """
    This function takes test_id and number of pages of workflow runs as input.
    Interacts with github api and collects the data for the tests with the specified id.
    Outputs the data frame with test data.
    """
    for p in range(1,page_numbers+1):
        command = """curl \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/actions/workflows/{}/runs?page={}""".format(TOKEN, ORG, REPO, test_id, p)
        args = []
        args.append(command)

        output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
        output = json.loads(output.stdout)

        if p==1:
            df = pd.json_normalize(output['workflow_runs'])
        else:
            df2 = pd.json_normalize(output['workflow_runs'])
            df = pd.concat([df, df2], axis=0)
    return df

In [6]:
def get_check_runs(commits):
    """
    This function takes test_id and number of pages of workflow runs as input.
    Interacts with github api and collects the data for the tests with the specified id.
    Outputs the data frame with test data.
    """
    appended_data = []
    for commit in commits:
        command = """curl -L \
          -H "Accept: application/vnd.github+json" \
          -H "Authorization: Bearer {}"\
          -H "X-GitHub-Api-Version: 2022-11-28" \
          https://api.github.com/repos/{}/{}/commits/{}/check-runs""".format(TOKEN, ORG, REPO, commit)
        args = []
        args.append(command)

        output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
        output = json.loads(output.stdout)

        appended_data.append(pd.json_normalize(output['check_runs']))

    df = pd.concat(appended_data, axis=0)

    return df

In [7]:
def get_commits():
    "This function gets all commit refs for a github repo"

    command =  """curl -L \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/commits""".format(TOKEN, ORG, REPO)
    args = []
    args.append(command)

    output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
    output = json.loads(output.stdout)

    df = pd.json_normalize(output)
    return df

In [8]:
if MODE == "check":
    # get checks
    df_commits = get_commits()
    commit_ids = list(df_commits['sha'])
    checks_df = get_check_runs(commit_ids)
    test_df = checks_df[['started_at', 'completed_at', 'id', 'status', 'conclusion', 'external_id', 'name']]
    test_df = test_df.rename(columns={'external_id': 'test_id', 'id': 'run_id'})
    test_df['run_duration'] = test_df.apply(lambda x: (datetime.strptime(x['completed_at'], "%Y-%m-%dT%H:%M:%SZ") - \
                                           datetime.strptime(x['started_at'], "%Y-%m-%dT%H:%M:%SZ")), axis = 1)

elif MODE == "workflow":
    # get workflows
    test_id = "28698040" # Pre-commit test
    page_numbers = get_page_numbers(test_id)
    workflow_df = get_workflow_runs(test_id, page_numbers)
    test_df = workflow_df[['created_at', 'updated_at', 'id', 'status', 'conclusion']]
    test_df = test_df.rename(columns={'id': 'run_id'})
    test_df['run_duration'] = test_df.apply(lambda x: (datetime.strptime(x['updated_at'], "%Y-%m-%dT%H:%M:%SZ") - \
                                           datetime.strptime(x['created_at'], "%Y-%m-%dT%H:%M:%SZ")), axis = 1)
    test_df['test_id'] = test_id

In [9]:
test_df.head(10)

,started_at,completed_at,run_id,status,conclusion,test_id,name,run_duration
0,2023-03-02T18:05:02Z,2023-03-02T18:05:07Z,11724147911,completed,success,6112eaac-d5ef-54f5-f329-09fa435f2cfc,containerfile-lint,0 days 00:00:05
1,2023-03-02T18:04:59Z,2023-03-02T18:05:04Z,11724147603,completed,success,826d253d-c1cf-53be-1b83-6be6927481a9,yaml-lint,0 days 00:00:05
2,2023-03-02T18:05:03Z,2023-03-02T18:05:19Z,11724147201,completed,success,607ecf98-37d3-5426-3fda-a220bdce326f,pre-commit,0 days 00:00:16
3,2023-03-02T18:05:02Z,2023-03-02T18:05:14Z,11724147098,completed,failure,0491c755-1a63-51b3-0919-c332635c76a6,lint-spelling,0 days 00:00:12
4,2023-03-02T18:05:02Z,2023-03-02T18:05:08Z,11724147087,completed,success,432fa4d3-634b-5861-14e7-b8c977033498,bash-lint,0 days 00:00:06
0,2023-03-01T21:58:24Z,2023-03-01T21:58:29Z,11699647668,completed,success,826d253d-c1cf-53be-1b83-6be6927481a9,yaml-lint,0 days 00:00:05
1,2023-03-01T21:58:24Z,2023-03-01T21:58:34Z,11699647500,completed,failure,0491c755-1a63-51b3-0919-c332635c76a6,lint-spelling,0 days 00:00:10
2,2023-03-01T21:58:21Z,2023-03-01T21:58:24Z,11699647486,completed,success,6112eaac-d5ef-54f5-f329-09fa435f2cfc,containerfile-lint,0 days 00:00:03
3,2023-03-01T21:58:21Z,2023-03-01T21:58:27Z,11699647274,completed,success,432fa4d3-634b-5861-14e7-b8c977033498,bash-lint,0 days 00:00:06
4,2023-02-28T22:21:34Z,2023-02-28T22:21:38Z,11669517832,completed,success,6112eaac-d5ef-54f5-f329-09fa435f2cfc,containerfile-lint,0 days 00:00:04


In [10]:
test_df.shape

(117, 8)

In [11]:
# generating passing and failing dfs which are neccesary for computing fit distributions
passing_test = test_df[test_df['conclusion'] == 'success'] 
failures_test = test_df[test_df['conclusion'] == 'failure'] 

In [12]:
passing_test.shape

(103, 8)

In [13]:
failures_test.shape

(14, 8)

## Conclusion

In this notebook, we interact with the github api to collect the data for all workflow runs. In future work, we will look into using this data to perform statistical tests using OSP model.